In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


## Variables

In [0]:
# variables
batch_size = 64
num_classes = 10
epochs = 200

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3).astype('float32')
x_test  = x_test.reshape(x_test.shape[0], 32, 32, 3).astype('float32')
x_train /= 255
x_test /= 255
print('x_train.shape[0]', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrixs
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)

x_train.shape[0] (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Model Architecture

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), border_mode='same', activation='relu', input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), border_mode='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
# model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3..., padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  after removing the cwd from sys.path.


In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)       

## Training

In [0]:
model.compile(loss = keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [7]:
model.fit(x_train, y_train, 
          batch_size=batch_size,
          epochs=epochs, 
          verbose=1, 
          validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/200
50000/50000 [==============================] - 23s 455us/step - loss: 1.4413 - accuracy: 0.4796 - val_loss: 1.4800 - val_accuracy: 0.5070
Epoch 2/200
50000/50000 [==============================] - 20s 396us/step - loss: 1.0186 - accuracy: 0.6480 - val_loss: 1.0583 - val_accuracy: 0.6551
Epoch 3/200
50000/50000 [==============================] - 20s 394us/step - loss: 0.8713 - accuracy: 0.7017 - val_loss: 0.9072 - val_accuracy: 0.6882
Epoch 4/200
50000/50000 [==============================] - 20s 397us/step - loss: 0.7824 - accuracy: 0.7337 - val_loss: 0.7514 - val_accuracy: 0.7430
Epoch 5/200
50000/50000 [==============================] - 20s 399us/step - loss: 0.7161 - accuracy: 0.7580 - val_loss: 0.8924 - val_accuracy: 0.7046
Epoch 6/200
50000/50000 [==============================] - 20s 398us/step - loss: 0.6669 - accuracy: 0.7754 - val_loss: 0.6724 - val_accuracy: 0.7710
Epoch 7/200
50000/50000 [=========================

- 2 NN >>> 73%
- 3 NN >>> 74.7% (1 dropout)
- 4 NN >>> 72.27% (1 dropout)
- 4 NN >>> 69.84% (2 dropout, test70%)

Under 4 NN
- Conv 32 64 64 >>> 66.60% (3 Maxpool)
- Conv 32 64 64 >>> 68% (2 Maxpool)

Under 3 NN
- Conv 32 64 64 >>> 70.43% (2 Maxpool)
- Conv 32 64 64 >>> 69.43% (1 Maxpool)
- Conv 16 32 64 >>> 68.14% (2 Maxpool)
- Conv 16 32 64 >>> 63.54% (1 Maxpool)
- Conv 16 32 64 128 >>> 68.04% (2 Maxpool)

50 epochs
- Conv 32 64 128 NN4 >>> 82.30% (3 Maxpool) 
- Conv 32 64 128 NN4 >>> 80.00% (2 Maxpool) **Overfit!**

100 epochs
- Conv 32 64 128 NN4 >>> 85.10% (3 Maxpool) 

200 epochs
- Conv 32 64 128 NN4 >>> 85.65% (3 Maxpool) 

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Test loss:  0.7004395915925503
Test accuracy:  0.8565000295639038


In [0]:
model.save('cifar10-200e-8565-addnormalize.model')